In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

import jellyfish
from elasticsearch import Elasticsearch
import json

In [2]:
import pandas as pd
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

In [85]:
f = open('config.txt')
config = json.load(f)
config_bc = sc.broadcast(config)
config

{'index_data': 'yes',
 'es_index_name': 'fd-cidacs-rl',
 'es_connect_string': 'http://localhost:9200',
 'query_size': 50,
 'cutoff_exact_match': '0.95',
 'datasets_info': {'indexed_dataset': {'path': '../0_global_data/fd-cidacs-rl/sinthetic-dataset-A.parquet',
   'extension': 'parquet',
   'columns': ['id_cidacs_a', 'nome_a', 'nome_mae_a', 'dt_nasc_a', 'sexo_a'],
   'id_column_name': 'id_cidacs_a'},
  'tolink_dataset': {'path': '../0_global_data/fd-cidacs-rl/sinthetic-datasets-b/sinthetic-datasets-b-1000.parquet',
   'extension': 'parquet',
   'columns': ['id_cidacs_b', 'nome_b', 'nome_mae_b', 'dt_nasc_b', 'sexo_b'],
   'id_column_name': 'id_cidacs_b'},
  'result_dataset': {'path': '../0_global_data/result/'}},
 'comparisons': {'name': {'indexed_col': 'nome_a',
   'tolink_col': 'nome_b',
   'must_match': 'true',
   'should_match': 'true',
   'is_fuzzy': 'true',
   'boost': '3.0',
   'query_type': 'match',
   'similarity': 'jaro_wikler',
   'weight': 1.0,
   'penalty': 0.02},
  'mothers

# ES functions

#### indexing

In [86]:
def index_dataframe(dataframe, es_index_name):
    # creating new index
    dataframe.write.format("org.elasticsearch.spark.sql") \
                 .option("es.resource", es_index_name).mode('overwrite').save()

#### exact query building

In [91]:
[config_['comparisons'][x]['indexed_col'] for x in config_['comparisons'] if config_['comparisons'][x]['must_match'] == 'true']

['nome_a', 'nome_mae_a', 'sexo_a']

In [92]:
def build_exact_queries(vars_col): 
    """
    Let us suppose the following values:
    vars_col = ['ROBESPIERRE PITA', '1987-05-05', '1', 'Mari Santos']
    indexed_cols = ['name', 'birthdate', 'sex', 'mothers_name']
    query_size = 10
    
    and only the first two attributes are assigned to exact match.
    So, the resulting query column would be: 
    '{ "size": "50", "query": 
                    { "bool": { "must": [ 
                                {"match": {"name":"ROBESPIERRE PITA"}},
                                {"birthdate": {"name":"1987-05-05"}}] } } }'
    Requirements: 
    - All values on vars_col must be converted into string
    - All the hyphens symbols must be taken from date type used to search (e.g. 1987-05-05 must be converted to 19870505)
    - The config json must be available as a broadcast through sc.broadcast() function.
    - The names of indexed columns must be correctly filled. 
    """
    config_ = config_bc.value
    query_size = config_['query_size']

    # getting names of indexed columns
    indexed_id_column = config_['datasets_info']['indexed_dataset']['id_column_name']
    
    indexed_cols = config_['datasets_info']['indexed_dataset']['columns']
    indexed_cols = [x for x in indexed_cols if x != indexed_id_column]
        
    # notice that we are linking indexed keys with tolink values
    # the keys will be used to set which field will be fetched on es
    # the values will be used as search content
    tolink_cols_dict = dict(zip(indexed_cols, vars_col))
    
    # finding which are the columns used on exact match step
    indexed_exact_match_vars = [config_['comparisons'][x]['indexed_col'] for x in config_['comparisons'] if config_['comparisons'][x]['must_match'] == 'true']
    non_exact_match_cols = list(set(indexed_cols) - set(indexed_exact_match_vars))
    # deleting those columns of non-exact match
    [tolink_cols_dict.pop(x, None) for x in non_exact_match_cols]
    
    # -------------------------------------------- #
    #   starting the building of query string      #
    # -------------------------------------------- #
    # setting the preffix and suffix of query core
    prefix_ = """{"match": {"""
    suffix_ = """}}"""
    
    # filling the query core with all indexed columns and values from vars_col
    strings = []
    for col in list(tolink_cols_dict.keys()):
        string = str(prefix_) + "\"" + str(col) + "\"" + ":" + "\"" +  str(tolink_cols_dict[col]) + "\"" + str(suffix_)
        print(string)
        strings.append(string)
    
    # building the query core. 
    # Should be like: {"match": {"name":"ROBESPIERRE PITA"}}, {"birthdate": {"name":"1987-05-05"}}
    line = ','.join(strings)
    
    # Finally the final query string
    complete_query = """{ "size": "%s", "query": { "bool": { "must": [ %s ] } } }""" % (query_size,line)
    
    return complete_query
udf_build_exact_queries = F.udf(build_exact_queries, StringType()) 

# Reading prepocessed datasets

In [78]:
# getting the auxiliary variables
data_ext = config['datasets_info']['indexed_dataset']['extension']
data_path = config['datasets_info']['indexed_dataset']['path']

# test the extension of the dataset to properly read it
if data_ext == 'csv':
    indexed_dataset = spark.read.csv(data_path, header=True)
elif data_ext == 'parquet':
    indexed_dataset = spark.read.parquet(data_path)
else:
    print("Please make sure the extension for this dataset is set as 'csv' or 'parquet'")

In [79]:
indexed_dataset.limit(3).toPandas()

,id_cidacs_a,nome_a,nome_mae_a,dt_nasc_a,sexo_a
0,1,YASMIM VITORIA MATIAS FONSECA,TACIANY DOS SANTOS,2007-11-22,2
1,2,PEDRO HENRIQUE MARTINS DE CARVALHO,FRANCILEIDE DOS SANTOS ALVES,2006-11-02,1
2,3,FABRICIO RODRIGUES DOS SANTOS,MARCELA MACHADO DA SILVA,2007-11-07,1


In [100]:
# getting the auxiliary variables
data_ext = config['datasets_info']['tolink_dataset']['extension']
data_path = config['datasets_info']['tolink_dataset']['path']

# test the extension of the dataset to properly read it
if data_ext == 'csv':
    tolink_dataset = spark.read.csv(data_path, header=True)
elif data_ext == 'parquet':
    tolink_dataset = spark.read.parquet(data_path)
else:
    print("Please make sure the extension for this dataset is set as 'csv' or 'parquet'")

#### preprocessing tolink dataset

In [102]:
for col in tolink_dataset.columns:
    tolink_dataset = tolink_dataset.withColumn(col, F.col(col).cast('string'))

# All the hyphens symbols must be taken from date type variables converted to string
tolink_dataset = tolink_dataset.withColumn('dt_nasc_b', F.regexp_replace(F.col('dt_nasc_b'), "-", ""))

In [103]:
tolink_dataset.limit(3).toPandas()

,id_cidacs_b,nome_b,nome_mae_b,dt_nasc_b,sexo_b
0,1081,FABIOLA FAGUNDES FRICKS,LUCIMARA COSTA NASCIMENTO,20070816,2
1,4582,ANA KAROLINA RODRIGUES SOUSA,CELINE RAIMUNDA SILVA,20090614,2
2,4739,NATALIA DAVID BENTO,KETLEN SANTOS,20091222,2


# Indexing dataset

In [80]:
# make sure all the cols in data are StringType()

for col in indexed_dataset.columns:
    indexed_dataset = indexed_dataset.withColumn(col, F.col(col).cast('string'))

# All the hyphens symbols must be taken from date type variables converted to string
indexed_dataset = indexed_dataset.withColumn('dt_nasc_a', F.regexp_replace(F.col('dt_nasc_a'), "-", ""))

# indexing, at last
index_df_response = config['index_data']
index_name = config['es_index_name']
if index_df_response == 'yes':
    index_dataframe(indexed_dataset, index_name)

In [70]:
# es = Elasticsearch('http://localhost:9200')
# content = {
#     'size': 1,
#     'query': {
#         'bool': {
#             'must': [
#                 {'match': {'dt_nasc_a': '20071122'}}
#             ]
#         }
#     }
# }
# es.search(index=index_name, body=content)

# Linking datasets

#### auxiliary variables

In [104]:
config_ = config_bc.value
query_size = config_['query_size']

#### creating vars column

In [105]:
tolink_id_column = config_['datasets_info']['tolink_dataset']['id_column_name']
tolink_cols = config_['datasets_info']['tolink_dataset']['columns']
tolink_cols = [x for x in tolink_cols if x != tolink_id_column]

tolink_dataset = tolink_dataset.withColumn('vars', F.array(tolink_cols))
tolink_dataset.limit(2).toPandas()

,id_cidacs_b,nome_b,nome_mae_b,dt_nasc_b,sexo_b,vars
0,1081,FABIOLA FAGUNDES FRICKS,LUCIMARA COSTA NASCIMENTO,20070816,2,"[FABIOLA FAGUNDES FRICKS, LUCIMARA COSTA NASCI..."
1,4582,ANA KAROLINA RODRIGUES SOUSA,CELINE RAIMUNDA SILVA,20090614,2,"[ANA KAROLINA RODRIGUES SOUSA, CELINE RAIMUNDA..."


#### creating exact_queries column

In [106]:
tolink_dataset = tolink_dataset.withColumn('exact_queries', udf_build_exact_queries(F.col('vars')))
tolink_dataset.limit(3).toPandas()

,id_cidacs_b,nome_b,nome_mae_b,dt_nasc_b,sexo_b,vars,exact_queries
0,1081,FABIOLA FAGUNDES FRICKS,LUCIMARA COSTA NASCIMENTO,20070816,2,"[FABIOLA FAGUNDES FRICKS, LUCIMARA COSTA NASCI...","{ ""size"": ""50"", ""query"": { ""bool"": { ""must"": [..."
1,4582,ANA KAROLINA RODRIGUES SOUSA,CELINE RAIMUNDA SILVA,20090614,2,"[ANA KAROLINA RODRIGUES SOUSA, CELINE RAIMUNDA...","{ ""size"": ""50"", ""query"": { ""bool"": { ""must"": [..."
2,4739,NATALIA DAVID BENTO,KETLEN SANTOS,20091222,2,"[NATALIA DAVID BENTO, KETLEN SANTOS, 20091222, 2]","{ ""size"": ""50"", ""query"": { ""bool"": { ""must"": [..."


#### finding the best candidate and similarity

In [153]:


indexed_id_column = config_['datasets_info']['indexed_dataset']['id_column_name']
tolink_id_column = config_['datasets_info']['tolink_dataset']['id_column_name']

indexed_cols = config_['datasets_info']['indexed_dataset']['columns']
indexed_cols = [x for x in indexed_cols if x != indexed_id_column]
print(indexed_cols)

tolink_cols = config_['datasets_info']['tolink_dataset']['columns']
tolink_cols = [x for x in tolink_cols if x != tolink_id_column]
print(tolink_cols)

['nome_a', 'nome_mae_a', 'dt_nasc_a', 'sexo_a']
['nome_b', 'nome_mae_b', 'dt_nasc_b', 'sexo_b']


In [154]:
list_values = ['FABIOLA FAGUNDES FRICKS', 'LUCIMARA COSTA NASCIMENTO', '2007-08-16', 2]

In [155]:
# notice that we are linking indexed keys with tolink values
tolink_cols_dict = dict(zip(indexed_cols, list_values))
tolink_cols_dict

{'nome_a': 'FABIOLA FAGUNDES FRICKS',
 'nome_mae_a': 'LUCIMARA COSTA NASCIMENTO',
 'dt_nasc_a': '2007-08-16',
 'sexo_a': 2}

In [156]:
indexed_exact_match_vars = [config_['comparisons'][x]['indexed_col'] for x in config_['comparisons'] if config_['comparisons'][x]['must_match'] == 'true']
indexed_exact_match_vars

['nome_a', 'nome_mae_a', 'sexo_a']

In [157]:
# tolink_exact_match_vars = [config_['comparisons'][x]['tolink_col'] for x in config_['comparisons'] if config_['comparisons'][x]['must_match'] == 'true']
# exact_match_vars

In [161]:
non_exact_match_cols = list(set(indexed_cols) - set(indexed_exact_match_vars))
[tolink_cols_dict.pop(x, None) for x in non_exact_match_cols]
tolink_cols_dict

{'nome_a': 'FABIOLA FAGUNDES FRICKS',
 'nome_mae_a': 'LUCIMARA COSTA NASCIMENTO',
 'sexo_a': 2}

In [135]:
query_size = config_['query_size']
print(type(query_size))
print(query_size)

<class 'int'>
50


In [136]:
prefix_ = """{"match": {"""
suffix_ = """}}"""

In [163]:
strings = []
for col in list(tolink_cols_dict.keys()):
    string = str(prefix_) + "\"" + str(col) + "\"" + ":" + "\"" +  str(tolink_cols_dict[col]) + "\"" + str(suffix_)
    print(string)
    strings.append(string)

{"match": {"nome_a":"FABIOLA FAGUNDES FRICKS"}}
{"match": {"nome_mae_a":"LUCIMARA COSTA NASCIMENTO"}}
{"match": {"sexo_a":"2"}}


In [164]:
line = ','.join(strings)
line

'{"match": {"nome_a":"FABIOLA FAGUNDES FRICKS"}},{"match": {"nome_mae_a":"LUCIMARA COSTA NASCIMENTO"}},{"match": {"sexo_a":"2"}}'

In [165]:
complete_query = """{ "size": "%s", "query": { "bool": { "must": [ %s ] } } }""" % (query_size,line)
complete_query

'{ "size": "50", "query": { "bool": { "should": [ {"match": {"nome_a":"FABIOLA FAGUNDES FRICKS"}},{"match": {"nome_mae_a":"LUCIMARA COSTA NASCIMENTO"}},{"match": {"sexo_a":"2"}} ] } } }'

In [ ]:
to_delete
tolink_cols_dict = 

In [ ]:
content = {
        'size': 5,
        'query': {
            'bool': {
                'must': [
                    {'match': {'lb_sex': sex}},
                    {'match': {'lb_birthday_child': birthday_child}},
                    {'match': {'lb_addr_residence': addr_residence}}
                ]
            }
        }
    }

In [ ]:
content = {
        'size': 5,
        'query': {
            'bool': {
                'should': [
                    {'match': {'lb_sex': {'query': sex, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2.0'}}},
                    {'match': {'lb_addr_residence': {'query': addr_residence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'4.0'}}},
                    {'match': {'lb_addr_occurrence': {'query': addr_occurrence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'lb_state_residence': {'query': state_residence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'1.0'}}},
                    {'match': {'lb_state_occurrence': {'query': state_occurrence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'lb_state_mun_residence': {'query': state_mun_residence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'3.0'}}},
                    {'match': {'lb_state_mun_occurrence': {'query': state_mun_occurrence, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'lb_day_birth': {'query': day_birth}}},
                    {'match': {'lb_month_birth': {'query': month_birth}}},
                    {'match': {'lb_year_birth': {'query': year_birth}}},
                    {'match': {'lb_mun_residence_pad': {'query': mun_residence_pad}}},
                    {'match': {'lb_loc_residence_pad': {'query': loc_residence_pad}}},
                    {'match': {'lb_mun_occurrence_pad': {'query': mun_occurrence_pad}}},
                    {'match': {'lb_loc_occurrence_pad': {'query': loc_occurrence_pad}}},
                    {'term': {'lb_birthday_child': birthday_child}}
                ]
            }
        }
    }

In [55]:
config['comparisons']

{'nome_a': {'compare_to': 'nome_b',
  'must_match': 'true',
  'should_match': 'true',
  'is_fuzzy': 'true',
  'boost': '3.0',
  'query_type': 'match',
  'similarity': 'jaro_wikler',
  'weight': 1.0,
  'penalty': 0.02},
 'nome_mae_a': {'compare_to': 'nome_mae_b',
  'must_match': 'true',
  'should_match': 'true',
  'is_fuzzy': 'true',
  'boost': '2.0',
  'query_type': 'match',
  'similarity': 'jaro_wikler',
  'weight': 1.0,
  'penalty': 0.02},
 'dt_nasc_a': {'compare_to': 'dt_nasc_b',
  'must_match': 'false',
  'should_match': 'true',
  'is_fuzzy': 'false',
  'boost': '',
  'query_type': 'term',
  'similarity': 'hamming',
  'weight': 1.0,
  'penalty': 0.02},
 'sexo_a': {'compare_to': 'sexo_b',
  'must_match': 'true',
  'should_match': 'true',
  'is_fuzzy': 'false',
  'boost': '',
  'query_type': 'term',
  'similarity': 'overlap',
  'weight': 1.0,
  'penalty': 0.02}}

In [ ]:
def build_queries(list_of_values_col, query_size):
    """
    Let us suppose the following column values:
    list_of_cols = ['ab', 'vbx']
    list_of_values = ['2', 'mamao']
    so, this udf must return a column with values like this: 
    {"match": {"ab":"2"}},{"match": {"vbx":"mamao"}}
    """
    query_size = str(query_size)
    list_of_cols = cols.value  # [x for x in cols.value if x != 'id']
    prefix_ = """{"match": {"""
    suffix_ = """}}"""
    strings = []
    dict_cols = dict(zip(list_of_cols, list_of_values_col))
    
    for col in list(dict_cols.keys()):
        string = str(prefix_) + "\"" + str(col) + "\"" + ":" + "\"" +  str(dict_cols[col]) + "\"" + str(suffix_)
        strings.append(string)
    
    line = ','.join(strings)
    complete_query = """{ "size": "%s", "query": { "bool": { "should": [ %s ] } } }""" % (query_size,line)
    return complete_query
udf_build_queries = F.udf(build_queries, StringType()) 